In [1]:
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(seed=42) #setting the seed so the random weights and biases our network chooses can be reporoduced

cwd = os.getcwd()

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator()



training_set = train_datagen.flow_from_directory(f'{cwd}/train', 
                                                 target_size=(64, 64), 
                                                 batch_size=32, 
                                                 class_mode='binary', 
                                                 seed=42)



test_set = test_datagen.flow_from_directory(f'{cwd}/test',
                                            target_size = (64, 64),
                                            batch_size = 1,
                                            class_mode = 'binary',
                                           shuffle=False)

Using TensorFlow backend.


Found 1000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

model = Sequential()

In [3]:
model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(128, (3, 3), activation = 'relu')) # change back to 128 and 3,3 from 512
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(units = 64, activation = 'relu')) # this was 64
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = 'sigmoid'))

In [4]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [5]:
model.fit_generator(training_set,epochs = 35, validation_data = test_set)

Epoch 1/35
32/32 [==============================] - 23s 715ms/step - loss: 0.6548 - accuracy: 0.6180 - val_loss: 0.0000e+00 - val_accuracy: 0.6900
Epoch 2/35
32/32 [==============================] - 18s 560ms/step - loss: 0.5951 - accuracy: 0.6980 - val_loss: 0.0000e+00 - val_accuracy: 0.7350
Epoch 3/35
32/32 [==============================] - 16s 514ms/step - loss: 0.5033 - accuracy: 0.7470 - val_loss: 0.0000e+00 - val_accuracy: 0.7600
Epoch 4/35
32/32 [==============================] - 17s 535ms/step - loss: 0.4734 - accuracy: 0.7810 - val_loss: 0.0000e+00 - val_accuracy: 0.8200
Epoch 5/35
32/32 [==============================] - 18s 574ms/step - loss: 0.4368 - accuracy: 0.8070 - val_loss: 0.0000e+00 - val_accuracy: 0.7950
Epoch 6/35
32/32 [==============================] - 18s 557ms/step - loss: 0.4211 - accuracy: 0.8140 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 7/35
32/32 [==============================] - 18s 556ms/step - loss: 0.3483 - accuracy: 0.8480 - val_loss: 0.000

In [ ]:
actual = []
predictions = []

# get actual values
for i in range(len(test_set)):
    actual.append(test_set[i][1][0]) #0 = Laurence Fishburne and 1=Samuel L Jackson
    
# get predicted values
for i in model.predict_generator(test_set):
    predictions.append(i[0])    

In [ ]:
# create a dataframe of both predicted and actual values
import pandas as pd
data = {'actual':actual, 'predictions':predictions} 
df = pd.DataFrame(data)
df['predictions'] = df['predictions'].apply(lambda prediction: 1 if prediction >= 0.5 else 0)
df.head()

In [ ]:
wrong = df[df['actual'] != df['predictions']]
print(f'Number of wrong predictions: {len(wrong)}')
wrong

In [ ]:
import matplotlib.pyplot as plt

for i in wrong.index:
    plt.imshow(np.uint8(test_set[i][0][0])) # change first number to change image
    plt.show()